In [1]:
# Importing libraries 
import os
import sys
import glob
import numpy as np
import scipy.optimize as op
import matplotlib.pyplot as plt
import urllib.request
from astropy.io import fits
from astropy.table import Table
from astropy.table import vstack
from astropy.io import fits
import pandas as pd
import urllib.request
import requests
import time

In [2]:
#Getting the individual coadded files names
local_dir = "/home/colgoat/spc_DESIedr/" #path to the catalogs
pn_sv1 = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv1/dark/" # File database link for sv1
pn_sv3 = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/sv3/dark/" # File database link for sv3
EDR_qsolist = [] # List of spectra of interest from DESI EDR
print("\n")
print("---------------------------------------------")
print("\n")
file1 = "QSO_cat_fuji_sv1_dark_bal_mask_v1.0.fits" # Name of the first catalog (sv1)
lc_sv1 = os.path.join(local_dir,file1) # lc_sv1 = Local file sv1
fc_sv1 = fits.open(lc_sv1) # fc_sv1 = Open .fits catalog file of sv1
SV1 = Table.read(fc_sv1) # SV1 = Reading the file of SV1 as a BinTable 
q_sv1 = (SV1['Z']>=2.1) & (SV1['PROGRAM']=='dark') # q_sv1 = Query from main catalog (filtering)
catfil_sv1 = SV1[q_sv1] # catfil_sv1 = Filtered catalog sv1
catfil_sv1_hpix = catfil_sv1.group_by("HPXPIXEL") # catfil_sv1 = Grouping filtered catalog by HEALPIX criteria
HPX_sv1 = catfil_sv1_hpix["HPXPIXEL"]/100 # HPX_sv1 = Getting the healpix ID for path on database
print("Number of spectra in SV1: ",len(catfil_sv1)) # Showing the number of targets on screen
for p,m in zip(HPX_sv1,catfil_sv1_hpix["HPXPIXEL"]): # Adding selected Targets to the list
    EDR_qsolist.append(pn_sv1+str(int(p))+"/"+str(m)+"/coadd-sv1-dark-"+str(m)+".fits")
fc_sv1.close()
file2 = "QSO_cat_fuji_sv3_dark_bal_mask_v1.0.fits" # Name of the first catalog (sv3)
lc_sv3 = os.path.join(local_dir,file2) # lc_sv3 = Local file sv3
fc_sv3 = fits.open(lc_sv3) # fc_sv3 = Open .fits catalog file of sv3
SV3 = Table.read(fc_sv3) # SV3 = Reading the file of SV3 as a BinTable
q_sv3 = (SV3['Z']>=2.1) & (SV3['PROGRAM']=='dark') # q_sv3 = Query from main catalog (filtering)
catfil_sv3 = SV3[q_sv3] # catfil_sv3 = Filtered catalog sv3
catfil_sv3_hpix = catfil_sv3.group_by("HPXPIXEL") # catfil_sv3 = Grouping filtered catalog by HEALPIX criteria
HPX_sv3 = catfil_sv3_hpix["HPXPIXEL"]/100 # HPX_sv3 = Getting the healpix ID for path on database
print("Number of spectra in SV3: ",len(catfil_sv3))
for p,m in zip(HPX_sv3,catfil_sv3_hpix["HPXPIXEL"]): # Adding selected Targets to the list
    EDR_qsolist.append(pn_sv3+str(int(p))+"/"+str(m)+"/coadd-sv3-dark-"+str(m)+".fits")
fc_sv3.close()
EDRcat_stk = vstack(([catfil_sv1,catfil_sv3])) # EDR filtered catalogs stacked for both surveys and sorted
EDRcat_stk.sort(['TARGETID','SURVEY'])
print("Total number of DESI EDR spectra (SV1 & SV3) to be used: ",len(EDR_qsolist))



---------------------------------------------




Number of spectra in SV1:  11853
Number of spectra in SV3:  14910
Total number of DESI EDR spectra (SV1 & SV3) to be used:  26763


## First Survey (SV1)

In [3]:
maincat = Table.read(fits.open('/home/colgoat/spc_DESIedr/zall-pix-edr-vac.fits')[1])
maincat = maincat[(maincat['PROGRAM']=='dark') & (maincat['SURVEY']=='sv1')] # | (cor_cat['SURVEY']=='sv3'))]
maincat.sort(['TARGETID'])
len(maincat)

374897

In [4]:
catfil_sv1.sort(['TARGETID'])
len(catfil_sv1)

11853

In [5]:
MC_TID = np.array(maincat['TARGETID']) # Main cat Target ID
SV1_TID = np.array(catfil_sv1['TARGETID']) # SV1 cat Target ID
A = np.intersect1d(MC_TID,SV1_TID)
mask1 = np.isin(MC_TID,A)
F1 = maincat[mask1]
len(F1)

11853

In [6]:
F1TID = np.array(F1['TARGETID'])
F1LN = np.array(F1['LASTNIGHT'])
data1 = {'MCID': F1TID, 'LN': F1LN, 'SV1ID': SV1_TID}
df1 = pd.DataFrame(data = data1)
df1

,MCID,LN,SV1ID
0,39627126193784363,20210212,39627126193784363
1,39627131495384271,20210212,39627131495384271
2,39627131507969517,20210212,39627131507969517
3,39627131507970322,20210212,39627131507970322
4,39627136847319542,20210212,39627136847319542
...,...,...,...
11848,616094317391381068,20210306,616094317391381068
11849,616094318045692832,20210204,616094318045692832
11850,616094319358509095,20210204,616094319358509095
11851,616094319379481599,20210306,616094319379481599


Sanity check!

In [7]:
a = 0
for i in range(len(df1)):
    if (df1['MCID'][i] == df1['SV1ID'][i]):
        a = a + 0
    else:
        a = a + 1
print(a)

0


Run once!

In [8]:
#catfil_sv1.colnames
#catfil_sv1.colnames[64] = 'PROGRAM'
catfil_sv1.add_column(data1['LN'], name='LASTNIGHT', index=65)

Verification for SV1

In [9]:
sv1_addcol = catfil_sv1
sv1_addcol

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,LASTNIGHT,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK,Z_MASK,ZERR_MASK,ZWARN_MASK,CHI2_MASK,DELTACHI2_MASK,SPECTYPE_MASK,SNR_CIV
,,,,,,deg,deg,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,,,,,,,,
int64,float64,float64,int64,str6,int32,float64,float64,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,str3,str4,int32,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8,float64,float64,int64,float64,float64,str6,float64
39627126193784363,2.861444156203475,0.0004390203909165424,0,QSO,0,163.34224841332946,-28.383592117072975,PSF,0.056114484,0.22035833,0.24633299,0.2883746,5.040667,3.8000672,1079.8296,834.0869,183.63834,3.4844775,0.7739383,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.2498812813325785,0,549755814402,0,0,19,17061.107,0,0,0,0,0,0,429.65128,131.00398,--,--,--,--,--,--,--,2.8614442,2.8628533,0.9959417,0.92074513,0.5857079,0.4896148,0.001158042,0.0001235028,2.8628533,2.8609123,2.1212556,2.1074882,0.36590034,0.05881108,10,38771,sv1,dark,20210212,3.7389505 .. -1.0375798,1.8037806,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.8195935774921809
39627131495384271,2.2484643217033646,0.0003227390116080642,0,GALAXY,0,161.91828131658428,-28.192263156157672,REX,0.062367868,0.38231152,0.6488977,1.5420872,2.10844,3.2716653,423.10953,321.8081,78.951126,3.4995875,0.77653664,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.7119825367890298,0,4123168608002,0,0,19,17061.107,0,0,0,0,0,0,129.02957,126.519226,2.0043762,103.66812,3941.6396,-103.52978,6101342400000.0,2206055300000000.0,6101343000000.0,0.82062626,2.2044537,0.00058004295,0.000117315016,0.98509276,0.35787344,1.9994082e-05,7.00337e-07,2.181541,2.2812526,2.2044537,2.2091928,0.5625591,0.061835185,24,38776,sv1,dark,20210212,7.030149 .. -0.04720141,1.5978751,0.0,0.0,0.0,0,-1.0 .. -1.0,

#### Done for SV1

## Second Survey (SV3)

In [10]:
maincat = Table.read(fits.open('/home/colgoat/spc_DESIedr/zall-pix-edr-vac.fits')[1])
maincat = maincat[(maincat['PROGRAM']=='dark') & (maincat['SURVEY']=='sv3')]
maincat.sort(['TARGETID'])
len(maincat)

699584

In [11]:
catfil_sv3.sort(['TARGETID'])
len(catfil_sv3)

14910

In [12]:
MC_TID = np.array(maincat['TARGETID']) # Main cat Target ID
SV3_TID = np.array(catfil_sv3['TARGETID']) # SV1 cat Target ID
B = np.intersect1d(MC_TID,SV3_TID)
mask2 = np.isin(MC_TID,B)
F2 = maincat[mask2]
len(F2)

14910

In [13]:
F2TID = np.array(F2['TARGETID'])
F2LN = np.array(F2['LASTNIGHT'])
data2 = {'MCID': F2TID, 'LN': F2LN, 'SV1ID': SV3_TID}
df2 = pd.DataFrame(data = data2)
df2

,MCID,LN,SV1ID
0,1083386135314440,20210507,1083386135314440
1,1083393307574272,20210505,1083393307574272
2,1084093743759365,20210416,1084093743759365
3,1084393787490304,20210429,1084393787490304
4,1084458555932679,20210419,1084458555932679
...,...,...,...
14905,616094218179314456,20210412,616094218179314456
14906,616094218179314845,20210406,616094218179314845
14907,616094218187702398,20210407,616094218187702398
14908,616094218191896647,20210411,616094218191896647


Sanity check!!

In [14]:
a = 0
for i in range(len(df2)):
    if (df2['MCID'][i] == df2['SV1ID'][i]):
        a = a + 0
    else:
        a = a + 1
print(a)

0


Run once!!

In [15]:
#catfil_sv1.colnames
#catfil_sv1.colnames[64] = 'PROGRAM'
catfil_sv3.add_column(data2['LN'], name='LASTNIGHT', index=65)

Verification for SV3

In [16]:
sv3_addcol = catfil_sv3
sv3_addcol

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV3_DESI_TARGET,SV3_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV1_DESI_TARGET,SV2_DESI_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,LASTNIGHT,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK,Z_MASK,ZERR_MASK,ZWARN_MASK,CHI2_MASK,DELTACHI2_MASK,SPECTYPE_MASK,SNR_CIV
,,,,,,deg,deg,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,,,,,,,,
int64,float64,float64,int64,str6,int32,float64,float64,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,str3,str4,int32,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8,float64,float64,int64,float64,float64,str6,float64
1083386135314440,3.631155815059421,0.0007740990556025613,4,QSO,0,150.1172290943953,3.203639817689895,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.997675796508789,0,4611686018427387904,70368744177664,0,4,5545.4507,0,0,0,0,0,0,205.45746,108.59245,--,--,--,--,--,--,--,3.6311557,1.5085001,0.0007476383,4.9172184e-05,0.0020246075,0.00030511015,0.00014915048,0.00031868834,3.9982815,4.1089396,1.5085001,0.32323426,0.33692527,0.22890207,2,27259,sv3,dark,20210507,0.38701344 .. -0.13898505,1.0797844,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.49071757215759687
1083393307574272,2.4328130904523575,0.0015506419586017728,4,GALAXY,0,218.11601975479252,3.456844830548416,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.997675796508789,0,4611686018427387904,70368744177664,0,1,1023.7778,0,0,0,0,0,0,33.64711,31.710672,3.9321053,0.11392317,16.187904,-0.008994076,0.0032281901,90.45671,0.00033598195,1.225154,2.4456637,0.011825093,0.018332023,0.6120433,0.98057455,2.510112e-05,1.2357961e-05,2.437483,2.504086,2.451604,2.4456637,0.48333326,0.46334296,24,25979,sv3,dark,20210505,3.6173832 .. -0.3486411,0.9208186,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.52385616

#### Done for SV3

## Stacking tables and saving as new catalog

In [17]:
bal_13_LNc = vstack(([sv1_addcol,sv3_addcol])) #BAL catalogs sv1 and sv3 with LASTNIGHT column added
len(bal_13_LNc)

26763

In [18]:
bal_13_LNc

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,LASTNIGHT,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK,Z_MASK,ZERR_MASK,ZWARN_MASK,CHI2_MASK,DELTACHI2_MASK,SPECTYPE_MASK,SNR_CIV
,,,,,,deg,deg,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,,,,,,,,
int64,float64,float64,int64,str6,int32,float64,float64,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,str3,str4,int32,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8,float64,float64,int64,float64,float64,str6,float64
39627126193784363,2.861444156203475,0.0004390203909165424,0,QSO,0,163.34224841332946,-28.383592117072975,PSF,0.056114484,0.22035833,0.24633299,0.2883746,5.040667,3.8000672,1079.8296,834.0869,183.63834,3.4844775,0.7739383,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.2498812813325785,0,549755814402,0,0,19,17061.107,0,0,0,0,0,0,429.65128,131.00398,--,--,--,--,--,--,--,2.8614442,2.8628533,0.9959417,0.92074513,0.5857079,0.4896148,0.001158042,0.0001235028,2.8628533,2.8609123,2.1212556,2.1074882,0.36590034,0.05881108,10,38771,sv1,dark,20210212,3.7389505 .. -1.0375798,1.8037806,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.8195935774921809
39627131495384271,2.2484643217033646,0.0003227390116080642,0,GALAXY,0,161.91828131658428,-28.192263156157672,REX,0.062367868,0.38231152,0.6488977,1.5420872,2.10844,3.2716653,423.10953,321.8081,78.951126,3.4995875,0.77653664,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.7119825367890298,0,4123168608002,0,0,19,17061.107,0,0,0,0,0,0,129.02957,126.519226,2.0043762,103.66812,3941.6396,-103.52978,6101342400000.0,2206055300000000.0,6101343000000.0,0.82062626,2.2044537,0.00058004295,0.000117315016,0.98509276,0.35787344,1.9994082e-05,7.00337e-07,2.181541,2.2812526,2.2044537,2.2091928,0.5625591,0.061835185,24,38776,sv1,dark,20210212,7.030149 .. -0.04720141,1.5978751,0.0,0.0,0.0,0,-1.0 .. -1.0,

In [19]:
hdu = fits.BinTableHDU(bal_13_LNc)

with fits.open('/home/colgoat/spc_DESIedr/catQSO_dark_bal_mask_sv13_wLN.fits', mode='update') as passer:
    passer[1] = hdu
    hdr = passer[1].header
    hdr['EXTNAME'] = 'CATALOG'
    passer.flush()  # changes are written back to original.fits

In [20]:
pruf=Table.read('/home/colgoat/spc_DESIedr/catQSO_dark_bal_mask_sv13_wLN.fits')
pruf

TARGETID,Z,ZERR,ZWARN,SPECTYPE,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,PROBA_RF,MASKBITS,SV1_DESI_TARGET,SV1_SCND_TARGET,DESI_TARGET,COADD_NUMEXP,COADD_EXPTIME,CMX_TARGET,SV2_DESI_TARGET,SV3_DESI_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SCND_TARGET,TSNR2_LYA,TSNR2_QSO,DELTA_CHI2_MGII,A_MGII,SIGMA_MGII,B_MGII,VAR_A_MGII,VAR_SIGMA_MGII,VAR_B_MGII,Z_RR,Z_QN,C_LYA,C_CIV,C_CIII,C_MgII,C_Hbeta,C_Halpha,Z_LYA,Z_CIV,Z_CIII,Z_MgII,Z_Hbeta,Z_Halpha,QSO_MASKBITS,HPXPIXEL,SURVEY,PROGRAM,LASTNIGHT,PCA_COEFFS,PCA_CHI2,BAL_PROB,BI_CIV,ERR_BI_CIV,NCIV_2000,VMIN_CIV_2000,VMAX_CIV_2000,POSMIN_CIV_2000,FMIN_CIV_2000,AI_CIV,ERR_AI_CIV,NCIV_450,VMIN_CIV_450,VMAX_CIV_450,POSMIN_CIV_450,FMIN_CIV_450,BI_SIIV,ERR_BI_SIIV,NSIIV_2000,VMIN_SIIV_2000,VMAX_SIIV_2000,POSMIN_SIIV_2000,FMIN_SIIV_2000,AI_SIIV,ERR_AI_SIIV,NSIIV_450,VMIN_SIIV_450,VMAX_SIIV_450,POSMIN_SIIV_450,FMIN_SIIV_450,BALMASK,Z_MASK,ZERR_MASK,ZWARN_MASK,CHI2_MASK,DELTACHI2_MASK,SPECTYPE_MASK,SNR_CIV
,,,,,,deg,deg,,mag,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,nanomaggy^-2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,km / s,km / s,,km / s,km / s,km / s,,,,,,,,,
int64,float64,float64,int64,bytes6,int32,float64,float64,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,int16,int64,int64,int64,int16,float32,int64,int64,int64,int64,int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64,bytes3,bytes4,int32,float32[5],float32,float32,float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],float32,float32,int32,float32[5],float32[5],float32[5],float32[5],float32,float32,int32,float32[17],float32[17],float32[17],float32[17],uint8,float64,float64,int64,float64,float64,bytes6,float64
39627126193784363,2.861444156203475,0.0004390203909165424,0,QSO,0,163.34224841332946,-28.383592117072975,PSF,0.056114484,0.22035833,0.24633299,0.2883746,5.040667,3.8000672,1079.8296,834.0869,183.63834,3.4844775,0.7739383,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.8287334193652452,0.2498812813325785,0,549755814402,0,0,19,17061.107,0,0,0,0,0,0,429.65128,131.00398,--,--,--,--,--,--,--,2.8614442,2.8628533,0.9959417,0.92074513,0.5857079,0.4896148,0.001158042,0.0001235028,2.8628533,2.8609123,2.1212556,2.1074882,0.36590034,0.05881108,10,38771,sv1,dark,20210212,3.7389505 .. -1.0375798,1.8037806,0.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1.0,-1.0,-1,-1.0,-1.0,-1,0.8195935774921809
39627131495384271,2.2484643217033646,0.0003227390116080642,0,GALAXY,0,161.91828131658428,-28.192263156157672,REX,0.062367868,0.38231152,0.6488977,1.5420872,2.10844,3.2716653,423.10953,321.8081,78.951126,3.4995875,0.77653664,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.8115644671584047,0.7119825367890298,0,4123168608002,0,0,19,17061.107,0,0,0,0,0,0,129.02957,126.519226,2.0043762,103.66812,3941.6396,-103.52978,6101342400000.0,2206055300000000.0,6101343000000.0,0.82062626,2.2044537,0.00058004295,0.000117315016,0.98509276,0.35787344,1.9994082e-05,7.00337e-07,2.181541,2.2812526,2.2044537,2.2091928,0.5625591,0.061835185,24,38776,sv1,dark,20210212,7.030149 .. -0.04720141,1.5978751,0.0,0.0,0.0,0,-1.

In [21]:
BALS = fits.open('/home/colgoat/spc_DESIedr/catQSO_dark_bal_mask_sv13_wLN.fits')
BALS.info()

Filename: /home/colgoat/spc_DESIedr/catQSO_dark_bal_mask_sv13_wLN.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  CATALOG       1 BinTableHDU    292   26763R x 105C   [K, D, D, K, 6A, J, D, D, 4A, E, E, E, E, E, E, E, E, E, E, E, D, D, D, D, D, D, I, K, K, K, I, E, K, K, K, K, K, K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, J, K, 3A, 4A, J, 5E, E, E, E, E, J, 5E, 5E, 5E, 5E, E, E, J, 17E, 17E, 17E, 17E, E, E, J, 5E, 5E, 5E, 5E, E, E, J, 17E, 17E, 17E, 17E, B, D, D, K, D, D, 6A, D]   
